<a href="https://colab.research.google.com/github/SanchiJoshi/Flag_Antenna_Model-/blob/main/Copy_of_flag_dataset_SVM_and_RF_withGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_excel(file_name)
data.head()

Saving flag dataset compressed.xlsx to flag dataset compressed.xlsx


,L,Lg1,Lg,Wg1,Wg2,Ls,Ws1,Wp,Frequency (GHz)
0,25.0,8.6,3.0,15.0,4.0,12.1,2.5,12.0,2.0263
1,25.0,8.6,3.1,15.0,4.0,12.1,2.5,12.0,2.0400
2,25.0,8.6,3.2,15.0,4.0,12.1,2.5,12.0,2.0263
3,25.0,8.6,3.3,15.0,4.0,12.1,2.5,12.0,2.0950
4,25.0,8.6,3.4,15.0,4.0,12.1,2.5,12.0,2.0950


In [3]:
X = data.drop(columns=['Frequency (GHz)'])
y = data['Frequency (GHz)']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 292
Test set size: 33


In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# **TRAINING SVR WITHOUT GENETIC ALGORITHM**

In [6]:
#training the model
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled, y_train)

SVR()

In [7]:
# Predicting on the test set
y_pred = svr_model.predict(X_test_scaled)

# Calculating evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")


Mean Squared Error (MSE): 0.00444036647949601
R² Score: 0.8915382345318348


In [119]:
# Define the new input data
new_data = np.array([[25, 8.6, 3, 15, 4, 12.1, 2.5, 12]])
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data)
# Predict the frequency
predicted_frequency = svr_model.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.1199731380426377


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [127]:
# Define the new input data
new_data = np.array([[26, 8.8, 5.2, 14.8, 4, 12.2, 3, 10]])
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data)
# Predict the frequency
predicted_frequency = svr_model.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.4008615092853716


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## **IMPLEMENTING GENETIC ALGORITHM**

In [8]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.7 MB/s eta 0:00:00


In [9]:
from sklearn.model_selection import cross_val_score
from deap import base, creator, tools, algorithms
import numpy as np
# Defining the fitness function for GA
def fitness_function(params):
    C, epsilon, gamma = params
    if C <= 0 or epsilon <= 0 or gamma <= 0:  # Ensure positive values
        return float('inf'),  # High fitness value for invalid parameters
    svr = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
    scores = cross_val_score(svr, X_train_scaled, y_train, cv=3, scoring='neg_mean_squared_error')
    return -np.mean(scores),  # Minimize negative MSE


In [10]:
# Create the individual and population
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize MSE
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Define attribute initialization ranges
toolbox.register("attr_C", np.random.uniform, 0.1, 100)  # C: [0.1, 100]
toolbox.register("attr_epsilon", np.random.uniform, 0.01, 1)  # epsilon: [0.01, 1]
toolbox.register("attr_gamma", np.random.uniform, 0.0001, 1)  # gamma: [0.0001, 1]

# Define individual and population
toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (toolbox.attr_C, toolbox.attr_epsilon, toolbox.attr_gamma),
    n=1,
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define GA operators
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)

# Custom mutation to enforce valid ranges
def custom_mutation(individual, indpb):
    individual[0] = max(0.1, min(100, individual[0]))  # C: [0.1, 100]
    individual[1] = max(0.01, min(1, individual[1]))  # epsilon: [0.01, 1]
    individual[2] = max(0.0001, min(1, individual[2]))  # gamma: [0.0001, 1]
    return individual,

toolbox.register("mutate", custom_mutation, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitness_function)


In [11]:
# Create an initial population
population = toolbox.population(n=30)  # Population size
ngen = 50                              # Number of generations
cxpb = 0.7                             # Crossover probability
mutpb = 0.2                            # Mutation probability

# Run the genetic algorithm
result, logbook = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=True)

# Extract the best individual
best_individual = tools.selBest(population, k=1)[0]
print("Best Parameters (C, epsilon, gamma):", best_individual)


gen	nevals
0  	30    
1  	18    
2  	23    
3  	30    
4  	20    
5  	22    
6  	26    
7  	22    
8  	24    
9  	24    
10 	24    
11 	26    
12 	23    
13 	16    
14 	19    
15 	20    
16 	23    
17 	25    
18 	23    
19 	21    
20 	23    
21 	24    
22 	17    
23 	22    
24 	23    
25 	24    
26 	21    
27 	23    
28 	20    
29 	23    
30 	26    
31 	22    
32 	18    
33 	29    
34 	26    
35 	26    
36 	21    
37 	25    
38 	25    
39 	17    
40 	21    
41 	24    
42 	27    
43 	20    
44 	26    
45 	22    
46 	22    
47 	24    
48 	23    
49 	25    
50 	16    
Best Parameters (C, epsilon, gamma): [38.39597201805377, 0.030247336001830633, 0.0012283439096120985]


In [12]:
# Train SVR with the best hyperparameters
optimized_svr = SVR(kernel='rbf', C=best_individual[0], epsilon=best_individual[1], gamma=best_individual[2])
optimized_svr.fit(X_train_scaled, y_train)

# Predict on test set
y_pred_optimized = optimized_svr.predict(X_test_scaled)

# Evaluate
mse_optimized = mean_squared_error(y_test, y_pred_optimized)
r2_optimized = r2_score(y_test, y_pred_optimized)

print(f"Optimized Mean Squared Error (MSE): {mse_optimized}")
print(f"Optimized R² Score: {r2_optimized}")

Optimized Mean Squared Error (MSE): 0.003872112945753128
Optimized R² Score: 0.9054185711634797


In [120]:
# Define the new input data
new_data = np.array([[25, 8.6, 3, 15, 4, 12.1, 2.5, 12]])
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data)
# Predict the frequency
predicted_frequency = optimized_svr.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.0547595208739016


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [128]:
# Define the new input data
new_data = np.array([[26, 8.8, 5.2, 14.8, 4, 12.2, 3, 10]])
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data)
# Predict the frequency
predicted_frequency = optimized_svr.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.3183484093236264


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


# **RANDOM FOREST WITHOUT GENETIC ALOGORITHM**


In [13]:
from sklearn.ensemble import RandomForestRegressor

In [14]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=5)
rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

In [15]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 0.0038633140444048063
R² Score: 0.9056334958501753


In [121]:
# Define the new input data
new_data = np.array([[25, 8.6, 3, 15, 4, 12.1, 2.5, 12]])
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data)
# Predict the frequency
predicted_frequency = rf_model.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.0330199999999987


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [129]:
# Define the new input data
new_data = np.array([[26, 8.8, 5.2, 14.8, 4, 12.2, 3, 10]])
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data)
# Predict the frequency
predicted_frequency = rf_model.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.3576580000000007


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
from sklearn.model_selection import cross_val_score
from deap import base, creator, tools, algorithms
import numpy as np

# **Random Forest with GA**

In [103]:
def rf_fitness(params):
  n_estimators, max_depth, min_samples_split = map(int, params)
  if n_estimators < 50 or max_depth < 5 or min_samples_split < 2:
    return float('inf'),
  rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, random_state=7)
  return -cross_val_score(rf, X_train_scaled, y_train, cv=3, scoring='neg_mean_squared_error').mean(),

In [112]:
toolbox = base.Toolbox()
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox.register("attr_n_estimators", lambda: np.random.randint(50, 150))
toolbox.register("attr_max_depth", lambda: np.random.randint(5, 30))
toolbox.register("attr_min_samples_split", lambda: np.random.randint(2, 8))
toolbox.register("individual", tools.initIterate, creator.Individual, lambda: [toolbox.attr_n_estimators(), toolbox.attr_max_depth(), toolbox.attr_min_samples_split()])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", rf_fitness)

In [113]:
# Run GA
population = toolbox.population(n=50)
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=100, verbose=True)
best = tools.selBest(population, k=1)[0]
print("Best Parameters:", best)

gen	nevals
0  	50    
1  	37    
2  	29    
3  	46    
4  	39    
5  	40    
6  	31    
7  	40    
8  	43    
9  	41    
10 	31    
11 	33    
12 	35    
13 	40    
14 	31    
15 	35    
16 	39    
17 	42    
18 	43    
19 	39    
20 	36    
21 	40    
22 	34    
23 	42    
24 	32    
25 	33    
26 	41    
27 	42    
28 	42    
29 	34    
30 	39    
31 	35    
32 	39    
33 	41    
34 	38    
35 	32    
36 	36    
37 	41    
38 	39    
39 	38    
40 	33    
41 	41    
42 	40    
43 	38    
44 	41    
45 	32    
46 	42    
47 	40    
48 	37    
49 	37    
50 	31    
51 	37    
52 	46    
53 	37    
54 	38    
55 	32    
56 	44    
57 	32    
58 	32    
59 	40    
60 	36    
61 	34    
62 	29    
63 	40    
64 	42    
65 	41    
66 	34    
67 	46    
68 	34    
69 	37    
70 	38    
71 	30    
72 	39    
73 	40    
74 	35    
75 	37    
76 	38    
77 	38    
78 	31    
79 	40    
80 	36    
81 	42    
82 	42    
83 	38    
84 	40    
85 	32    
86 	42    
87 	41    
88 	35    
89 	31    

In [114]:
# Train optimized model
optimized_rf = RandomForestRegressor(n_estimators=int(best[0]), max_depth=int(best[1]), min_samples_split=int(best[2]), random_state=19)
optimized_rf.fit(X_train_scaled, y_train)

RandomForestRegressor(max_depth=24, min_samples_split=3, n_estimators=75,
                      random_state=19)

In [115]:
# Evaluate
y_pred = optimized_rf.predict(X_test_scaled)
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"R²: {r2_score(y_test, y_pred)}")

MSE: 0.0038208716954943366
R²: 0.9066702057962354


In [116]:
# Define the new input data
new_data = np.array([[25, 8.6, 3, 15, 4, 12.1, 2.5, 12]])
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data)
# Predict the frequency
predicted_frequency = optimized_rf.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.0342651904761904


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [130]:
# Define the new input data
new_data = np.array([[26, 8.8, 5.2, 14.8, 4, 12.2, 3, 10]])
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data)
# Predict the frequency
predicted_frequency = optimized_rf.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.348890355555555


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [126]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Actual frequency value
actual_frequency = np.array([2.0263])
# Predicted values from different models
predictions = {
    "SVR without GA": np.array([2.1199]),
    "SVR with GA": np.array([2.0547]),
    "RF without GA": np.array([2.0330]),
    "RF with GA": np.array([2.0342])
}
# Compute and display MSE for each model
for model, pred in predictions.items():
    mse = mean_squared_error(actual_frequency, pred)
    print(f"{model} - MSE: {mse:.6f}")

SVR without GA - MSE: 0.008761
SVR with GA - MSE: 0.000807
RF without GA - MSE: 0.000045
RF with GA - MSE: 0.000062
